# AutoGen Core: Distributed Runtime with gRPC

This notebook demonstrates **distributed agent execution** using AutoGen Core's gRPC-based runtime. Unlike standalone runtimes (which run all agents in a single process), distributed runtimes enable:
1. **Horizontal scaling**: Agents run across multiple worker processes
2. **Fault isolation**: Agent failures don't crash the entire system
3. **Resource optimization**: Different agents can run on different hardware

**Architecture:** A central host coordinates message routing between distributed worker runtimes, each hosting one or more agents.

In [ ]:
# Import dependencies
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost, GrpcWorkerAgentRuntime
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown
from dotenv import load_dotenv

load_dotenv(override=True)

# Runtime Configuration
ALL_IN_ONE_WORKER = False  # Set to True for single-worker testing

## Phase 1: Define Message Schema

In [ ]:
@dataclass
class Message:
    content: str

## Phase 2: Initialize gRPC Host

In [ ]:
# Start gRPC Host (Message Router)
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start()

## Phase 3: Configure Tools

In [ ]:
# Web Search Tool
serper = GoogleSerperAPIWrapper()
langchain_serper = Tool(
    name="internet_search", 
    func=serper.run, 
    description="Search the internet"
)
autogen_serper = LangChainToolAdapter(langchain_serper)

In [ ]:
# Task Instructions
instruction1 = """Research and briefly list the pros of using AutoGen for an AI agent project."""
instruction2 = """Research and briefly list the cons of using AutoGen for an AI agent project."""

judge_prompt = """You must decide whether to use AutoGen for a project based on the following research. 
Provide your decision and brief rationale."""

## Phase 4: Define Distributed Agents

In [ ]:
# Research Agent 1 (Pros)
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(
            name, 
            model_client=model_client, 
            tools=[autogen_serper], 
            reflect_on_tool_use=True
        )

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

# Research Agent 2 (Cons)
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(
            name, 
            model_client=model_client, 
            tools=[autogen_serper], 
            reflect_on_tool_use=True
        )

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

# Judge Agent (Orchestrator)
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        # Request research from both agents
        response1 = await self.send_message(Message(content=instruction1), AgentId("player1", "default"))
        response2 = await self.send_message(Message(content=instruction2), AgentId("player2", "default"))
        
        # Synthesize decision
        result = f"## Pros:\n{response1.content}\n\n## Cons:\n{response2.content}\n\n"
        judgement = f"{judge_prompt}\n{result}Your decision:"
        text_msg = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([text_msg], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)

## Phase 5: Deploy Agents to Distributed Workers

In [ ]:
if ALL_IN_ONE_WORKER:
    # Single worker (for testing)
    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))
else:
    # Distributed: Each agent on separate worker
    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))

agent_id = AgentId("judge", "default")

## Execute Distributed Workflow

In [ ]:
# Trigger Workflow
response = await worker.send_message(Message(content="Go!"), agent_id)
display(Markdown(response.content))

In [ ]:
# Cleanup
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()
await host.stop()